In [1]:
import cv2
from ultralytics import YOLO
from collections import defaultdict

# Load the YOLO model
model = YOLO('yolov8n.pt')

In [2]:
class_list = model.names 
#class_list

In [3]:
class_list

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [4]:
# Open the video file
#cap = cv2.VideoCapture('TRAFFIC VEDIO.mp4')

In [5]:
!pip install lap --upgrade

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 18.3 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 14.2 MB/s  0:00:00
  Attempting uninstall: lap
    Found existing installation: lap 0.5.12
    Uninstalling lap-0.5.12:
      Successfully uninstalled lap-0.5.12



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import lap
print("lap installed successfully ✅")


lap installed successfully ✅


In [8]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from collections import defaultdict

# -------------------------
# USER INPUT
# -------------------------
INPUT_PATH = input("Enter image or video path: ").strip().replace('"', '').replace("'", '')
if not os.path.exists(INPUT_PATH):
    raise ValueError("❌ Invalid path")

# -------------------------
# CONFIG
# -------------------------
MODEL_PATH = "yolov8n.pt"
FRAME_SKIP = 2
RESIZE_WIDTH = 960
CLASSES = [1, 2, 3, 5, 6, 7]     # vehicle classes
LINE_Y = 430                    # horizontal counting line (Y)

IMAGE_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".webp")
VIDEO_EXTS = (".mp4", ".avi", ".mov", ".mkv")

# -------------------------
# LOAD MODEL
# -------------------------
model = YOLO(MODEL_PATH)
class_list = model.names

# -------------------------
# DATA STRUCTURES
# -------------------------
class_counts = defaultdict(int)
crossed_ids = set()
output_data = []

# =========================
# IMAGE MODE
# =========================
if INPUT_PATH.lower().endswith(IMAGE_EXTS):

    img = cv2.imread(INPUT_PATH)
    if img is None:
        raise ValueError("❌ Cannot read image")

    h, w, _ = img.shape
    scale = RESIZE_WIDTH / w
    img = cv2.resize(img, (RESIZE_WIDTH, int(h * scale)))

    results = model(img, classes=CLASSES, verbose=False)

    if results[0].boxes is not None:
        boxes = results[0].boxes.xyxy
        class_indices = results[0].boxes.cls.int()
        confidences = results[0].boxes.conf

        for box, cls_idx, conf in zip(boxes, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            class_name = class_list[int(cls_idx)]

            class_counts[class_name] += 1
            output_data.append({
                "Class": class_name,
                "Confidence": float(conf),
                "Source": "Image"
            })

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, class_name, (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0, 255, 255), 2)

    # SHOW COUNTS
    y = 30
    for cls, cnt in class_counts.items():
        cv2.putText(img, f"{cls}: {cnt}", (20, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        y += 30

    cv2.imshow("YOLO Image Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# =========================
# VIDEO MODE
# =========================
elif INPUT_PATH.lower().endswith(VIDEO_EXTS):

    cap = cv2.VideoCapture(INPUT_PATH)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % FRAME_SKIP != 0:
            continue

        h, w, _ = frame.shape
        scale = RESIZE_WIDTH / w
        frame = cv2.resize(frame, (RESIZE_WIDTH, int(h * scale)))

        results = model.track(
            frame,
            persist=True,
            tracker="bytetrack.yaml",
            classes=CLASSES,
            device="cpu",
            verbose=False
        )

        if results[0].boxes.id is None:
            cv2.imshow("YOLO Traffic Counter", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue

        boxes = results[0].boxes.xyxy
        track_ids = results[0].boxes.id.int()
        class_indices = results[0].boxes.cls.int()
        confidences = results[0].boxes.conf

        # 🔴 FULL-WIDTH RED LINE
        cv2.line(frame, (0, LINE_Y), (frame.shape[1], LINE_Y), (0, 0, 255), 3)

        for box, track_id, cls_idx, conf in zip(
            boxes, track_ids, class_indices, confidences
        ):
            x1, y1, x2, y2 = map(int, box)
            cy = (y1 + y2) // 2

            class_name = class_list[int(cls_idx)]
            track_id = int(track_id)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name} {track_id}",
                        (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 255, 255), 2)

            # 🔥 COUNT WHEN CROSSING ANYWHERE
            if cy > LINE_Y and track_id not in crossed_ids:
                crossed_ids.add(track_id)
                class_counts[class_name] += 1

                output_data.append({
                    "Track_ID": track_id,
                    "Class": class_name,
                    "Confidence": float(conf),
                    "Frame_No": frame_count,
                    "Source": "Video"
                })

        # 🔢 COUNTS TOP-LEFT
        y = 30
        for cls, cnt in class_counts.items():
            cv2.putText(frame, f"{cls}: {cnt}", (20, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                        (0, 255, 0), 2)
            y += 30

        cv2.imshow("YOLO Traffic Counter", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

else:
    raise ValueError("❌ Unsupported file type")

# =========================
# FINAL OUTPUT
# =========================
df_output = pd.DataFrame(output_data)

print("\n========== FINAL SUMMARY ==========")
print("Total Counted Objects:", len(df_output))
if "Track_ID" in df_output.columns:
    print("Unique Objects:", df_output["Track_ID"].nunique())

print("\nClass-wise Count:")
print(df_output["Class"].value_counts())

print("\n========== OUTPUT TABLE ==========")
df_output


Enter image or video path:  "C:\Users\HP\DEEP_FACE\TRAFFIC VEDIO.mp4"



========== FINAL SUMMARY ==========
Total Counted Objects: 35
Unique Objects: 35

Class-wise Count:
Class
car           31
truck          2
bus            1
motorcycle     1
Name: count, dtype: int64

========== OUTPUT TABLE ==========


,Track_ID,Class,Confidence,Frame_No,Source
0,2,car,0.853242,2,Video
1,1,car,0.893635,44,Video
2,3,car,0.921199,80,Video
3,4,car,0.910018,108,Video
4,7,car,0.726433,144,Video
5,6,bus,0.749546,188,Video
6,8,car,0.902918,204,Video
7,59,car,0.464282,214,Video
8,5,car,0.911174,380,Video
9,14,car,0.691540,512,Video
